In [1]:
import pandas as pd
from surprise import Reader, Dataset, SVD, evaluate
import warnings; warnings.simplefilter('ignore')

In [2]:
data = pd.read_csv("dataset/movies_metadata.csv")
credits = pd.read_csv("dataset/credits.csv")
keywords = pd.read_csv("dataset/keywords.csv")
data = data.merge(credits, on='id')
data = data.merge(keywords, on='id')
data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
4,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...","[{'id': 642, 'name': 'robbery'}, {'id': 703, '..."


In [3]:
def get_genres(x):
    genre = []
    for i in x:
        genre.append(i['name'])
    return genre

In [4]:
def get_cast(x):
    cast = []
    num = 0
    for i in x:
        i['name'] = i['name'].replace(" ","")
        cast.append(i['name'])
        num = num + 1
        if num==5:
            break
    return cast

In [5]:
def get_crew(x):
    crew = []
    for i in x:
        if i['job'] == 'Director' or i['job'] =='Screenplay':
            i['name'] = i['name'].replace(" ","")
            crew.append(i['name'])
    return crew

In [6]:
def get_keywords(x):
    keywords = []
    for i in x:
        i['name'] = i['name'].replace(" ","")
        keywords.append(i['name'])
    return keywords

In [7]:
from ast import literal_eval

data['genres'] = data['genres'].apply(literal_eval).apply(get_genres)
data['cast'] = data['cast'].apply(literal_eval).apply(get_cast)
data['crew'] = data['crew'].apply(literal_eval).apply(get_crew)
data['keywords'] = data['keywords'].apply(literal_eval).apply(get_keywords)

data['soup'] = data['genres']+data['cast']+data['crew']+data['keywords']
data['soup'] = data['soup'].apply(lambda x: ' '.join(x))

data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords,soup
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,"[TomHanks, TimAllen, DonRickles, JimVarney, Wa...","[JohnLasseter, JossWhedon, AndrewStanton, Joel...","[jealousy, toy, boy, friendship, friends, riva...",Animation Comedy Family TomHanks TimAllen DonR...
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[RobinWilliams, JonathanHyde, KirstenDunst, Br...","[JonathanHensleigh, JoeJohnston, GregTaylor, J...","[boardgame, disappearance, basedonchildren'sbo...",Adventure Fantasy Family RobinWilliams Jonatha...
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[WalterMatthau, JackLemmon, Ann-Margret, Sophi...",[HowardDeutch],"[fishing, bestfriend, duringcreditsstinger, ol...",Romance Comedy WalterMatthau JackLemmon Ann-Ma...
3,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[SteveMartin, DianeKeaton, MartinShort, Kimber...","[NancyMeyers, AlbertHackett, CharlesShyer]","[baby, midlifecrisis, confidence, aging, daugh...",Comedy SteveMartin DianeKeaton MartinShort Kim...
4,False,NaN,60000000,"[Action, Crime, Drama, Thriller]",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,"[AlPacino, RobertDeNiro, ValKilmer, JonVoight,...","[MichaelMann, MichaelMann]","[robbery, detective, bank, obsession, chase, s...",Action Crime Drama Thriller AlPacino RobertDeN...


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
count = CountVectorizer(analyzer='word', stop_words='english')
matrix = count.fit_transform(data['soup'])
similarity = cosine_similarity(matrix,matrix)

In [9]:
ratings = pd.read_csv("dataset/ratings.csv")
ratings.head()

,userId,movieId,rating
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1246,5.0
4,1,1968,4.0


In [10]:
reader = Reader()
dataset = Dataset.load_from_df(ratings, reader)
dataset.split(n_folds=5)
svd = SVD()
evaluate(svd, dataset, measures=['RMSE','MAE'])

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.9241
MAE:  0.7118
------------
Fold 2
RMSE: 0.8937
MAE:  0.6950
------------
Fold 3
RMSE: 0.9156
MAE:  0.7069
------------
Fold 4
RMSE: 0.9144
MAE:  0.7218
------------
Fold 5
RMSE: 0.9403
MAE:  0.7345
------------
------------
Mean RMSE: 0.9176
Mean MAE : 0.7140
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.9240643322105614,
                             0.89371663961893,
                             0.915569295328132,
                             0.9144335223137263,
                             0.9403382716861497],
                            'mae': [0.711816322813132,
                             0.6949756068197072,
                             0.7068873931559597,
                             0.7217734118989446,
                             0.7344709196779191]})

In [11]:
trainset = dataset.build_full_trainset()
svd.train(trainset)

In [12]:
titles = data[['title','vote_count','vote_average','id']]

In [13]:
def get_movie_list(movieId):
    score = list(enumerate(similarity[movieId]))
    score = sorted(score, key=lambda x: x[1], reverse=True)
    score = score[0:11]
    movie_indices = [i[0] for i in score]
    return movie_indices

In [14]:
def get_recommendation(userId):
    userdata = ratings[ratings['userId'] == userId]
    userdata = userdata[userdata['rating'] >= 2.5]
    if userdata.empty == True:
        return """Insufficient Data Present"""
    
    movie = set()
    for i in userdata['movieId']:
        movies = get_movie_list(i)
        for x in movies:
            movie.add(x)
    for i in userdata['movieId']:
        movie.remove(i)
        
    movie_indices = list(movie)
    top = pd.DataFrame(titles.iloc[movie_indices])
    top['rating'] = top['id'].apply(lambda x: svd.predict(userId, x).est)
    top = top.sort_values('rating', ascending=False)
    return top.head(10)

In [15]:
get_recommendation(1)

,title,vote_count,vote_average,id,rating
1536,The Funhouse,69.0,5.8,13555,3.585888
1245,The Texas Chain Saw Massacre,617.0,7.1,30497,3.585888
67,Rumble in the Bronx,245.0,6.5,33542,3.585888
3653,Police Story,128.0,7.3,9056,3.585888
3655,Spies Like Us,180.0,5.9,9080,3.585888
2895,Oscar,128.0,6.2,11890,3.585888
3024,Into the Night,59.0,6.4,11338,3.585888
3409,Napoleon Dynamite,567.0,6.7,8193,3.585888
9168,Banana,139.0,7.2,54551,3.585888
8148,The Lady in the Van,225.0,6.0,328589,3.585888


In [16]:
get_recommendation(2)

,title,vote_count,vote_average,id,rating
1381,The Killing,290.0,7.7,247,3.686817
709,Batman Returns,1706.0,6.6,364,3.591859
2152,Gentlemen Prefer Blondes,169.0,7.1,759,3.530491
1371,Eyes Wide Shut,1266.0,7.1,345,3.522481
431,Sabrina,284.0,7.4,6620,3.517477
1385,Barry Lyndon,521.0,7.7,3175,3.445224
3978,Broken Flowers,359.0,6.8,308,3.435043
2757,25th Hour,640.0,7.2,1429,3.350642
1377,Inspector Gadget,322.0,4.3,332,3.340264
1382,Killer's Kiss,101.0,6.2,10056,3.333237


In [17]:
get_recommendation(3)

,title,vote_count,vote_average,id,rating
3215,The Dreamers,680.0,7.2,1278,3.708416
522,Dial M for Murder,539.0,7.9,521,3.269596
879,Mr. Nice Guy,129.0,6.2,10622,3.269596
8081,Love,449.0,5.2,292431,3.269596
3461,A Short Film About Killing,58.0,7.6,10754,3.269596
379,Stealing Beauty,99.0,6.6,14553,3.269596
8570,My Big Fat Greek Wedding 2,328.0,5.6,302688,3.269596
8053,Southpaw,2112.0,7.3,307081,3.269596
3436,Killing Me Softly,80.0,5.6,14365,3.269596
920,On the Waterfront,368.0,8.0,654,3.269596


In [18]:
get_recommendation(4)

,title,vote_count,vote_average,id,rating
2064,Sweet November,292.0,6.4,1921,4.432027
1689,The Searchers,332.0,7.7,3114,4.374819
3978,Broken Flowers,359.0,6.8,308,3.943911
3023,The Rookie,134.0,5.6,6723,3.870220
2167,Kiss of the Dragon,304.0,6.4,2140,3.834885
904,Lethal Weapon 4,782.0,6.3,944,3.825769
3969,Bad News Bears,73.0,5.7,13341,3.825769
2943,Open Range,268.0,7.0,2055,3.825769
8571,Born to Be Blue,58.0,6.7,327528,3.825769
8570,My Big Fat Greek Wedding 2,328.0,5.6,302688,3.825769


In [19]:
get_recommendation(5)    

,title,vote_count,vote_average,id,rating
3145,A Shot in the Dark,115.0,7.2,1594,4.099555
4470,TMNT,349.0,6.0,1273,4.026387
1645,The Bodyguard,681.0,6.2,619,3.880541
6149,Columbus Circle,79.0,5.9,92000,3.716659
3876,Diary of a Mad Black Woman,71.0,6.4,16186,3.716659
2895,Oscar,128.0,6.2,11890,3.716659
331,Hellraiser: Bloodline,111.0,4.9,8766,3.716659
2359,Corky Romano,54.0,4.1,17708,3.716659
2358,Bandits,307.0,6.2,3172,3.716659
4392,Code Name: The Cleaner,78.0,4.7,14396,3.716659
